In [1]:
import importlib
import os
import re
import sys
from collections import defaultdict, namedtuple
from dataclasses import dataclass, fields, asdict
from pathlib import Path

import nibabel as nib
import numpy as np

In [2]:
PREPROC_DIR = "/home/srs-9/Dev/ms_mri/monai/preproc"
sys.path.append(PREPROC_DIR)
import data_file_manager as datafm
import hemond_data as hd
from record import DataSet

In [3]:
importlib.reload(datafm)
importlib.reload(hd)
importlib.reload(sys.modules['record']).DataSet

record.DataSet

## Hemond Data

In [4]:
@dataclass(slots=True)
class Scan:
    subid: int
    date: int
    image: Path = None
    label: Path = None
    
    def has_label(self):
        if self.label is not None:
            return True
        else:
            return False

    @classmethod
    def _field_names(cls):
        return cls.__slots__

In [7]:
data_dir = Path("/mnt/e/Data/Hemond/flair")
dataset = hd.HaemondData('hemond_data', Scan._field_names())

images = [
        file.name for file in os.scandir(data_dir) if file.name.split(".")[-1] == "gz"
    ]

subj, ses = hd.get_subj_ses(images[0])
check = dict(subid=subj, date=ses, image=images[0])
check

TypeError: DataSet.__init__() missing 2 required positional arguments: 'recordname' and 'fields'

In [6]:
dataset.add_record(dict(subid=subj, date=ses, image=images[0]))
dataset.records

{'subid': '1001', 'date': '20170215', 'image': 'sub-ms1001_ses-20170215.nii.gz'}


[hemond_data(subid='1001', date='20170215', image='sub-ms1001_ses-20170215.nii.gz', label='')]

In [ ]:
fileobjs = list(os.scandir(data_dir))
scans = [file.name for file in os.scandir(data_dir) if file.name.split('.')[-1]=='gz']
labels = [file.name for file in os.scandir(data_dir/"labels") if file.name.split('.')[-1]=='gz']

def get_subj_ses(filename):
    restr = re.compile(r"sub-ms(\d{4})_ses-(\d{8})\.nii\.gz")
    rematch = restr.match(filename)
    return rematch[1], rematch[2]

In [ ]:
Scan = namedtuple("Scan", ("subid", "date", "image", "label"), defaults=[0])
test = Scan(subid=4, date=3)

In [ ]:
thoo.pop('three', dict)

## Lesjak Data

In [ ]:
basepath = Path("/mnt/f/Data/ms_mri/lesjak_2017")
data_dir = basepath / "data"

lesjak_datafm = datafm.LesjakData(data_dir)
subjects = lesjak_datafm.subjects
modalities = ["FLAIR", "T1W", "T1WKS", "T2W"]
lesjak_datafm.get_scan(subjects[0], modalities[1])

In [ ]:
def load_scan(scan_path: Path) -> np.ndarray:
    img = nib.load(scan_path)
    return np.array(img.dataobj)

In [ ]:
subject = subjects[1]
imgs = [nib.load(lesjak_datafm.get_scan(subject, mod)) for mod in modalities]

In [ ]:
imgs[3].affine

In [ ]:
scan_paths = [lesjak_datafm.get_scan(subject, mod) for mod in modalities]
scans = [load_scan(path) for path in scan_paths]

In [ ]:
labels = datafm.assign_train_test(subjects, 0.2)

In [ ]:
basepath = Path("/mnt/f/Data/ms_mri/Brats_3d_seg/Task01_BrainTumour")
imagesTr = basepath / "imagesTr"
image_files = os.scandir(imagesTr)
image_paths = [file.path for file in image_files if "._" not in file.name]

In [ ]:
img = nib.load(image_paths[0])

a = np.array(img.dataobj)

In [ ]:
test = tuple()
if not test:
    print("uninitialzed")

In [ ]:
import random
fraction_ts = 0.2

n_subj = len(subjects)
n_ts = int(n_subj * fraction_ts)

random.shuffle(subjects)
subj_ts = [(subj, "ts") for subj in subjects[:n_ts]]
subj_tr = [(subj, "tr") for subj in subjects[n_ts:]]

subj_ts + subj_tr